In [17]:
import pandas as pd
import re
import random
from nltk import ngrams
from difflib import get_close_matches as gcm

In [2]:
df_ds = pd.read_csv('indeed-insights/data_scientist.csv')
df_ds.head()

,title,company,country,date_posted,description,title_keywords,indeed_skills,skills,no_skills
0,Data Scientist,Mission Consultancy Services Malaysia Sdn Bhd,Malaysia,2020-07-26,Identify valuable data sources and automate co...,NaN,"'C', 'C++', 'Excel', 'Hadoop', 'Java', 'R', 'S...","['Business', 'Analytical', 'Communication', 'D...",27
1,Data Scientist,AirAsia,Malaysia,2020-08-15,Job DescriptionOverviewThis role will responsi...,NaN,"'C', 'R'","['Business', 'Accuracy', 'Processing', 'Produc...",21
2,Data Scientist,CareerMaster Sdn Bhd,Malaysia,2020-07-26,Position to be based in Kuala Lumpur.We are re...,NaN,"'C', 'GIS', 'Go', 'Python', 'R', 'SQL', 'Stati...","['Business', 'Computer Science', 'Microsoft', ...",30
3,Data Scientist Executive,GENO Management,Malaysia,2020-08-18,Position : Data Scientists ExecutiveLocation :...,NaN,"'AWS', 'C', 'Go', 'Python', 'R', 'Regression',...","['Business', 'Data Science', 'Analytical', 'Bu...",39
4,Data Scientist,PLUS SOLAR SYSTEMS SDN BHD,Malaysia,2020-07-26,Work With Stakeholders Throughout The Organiza...,NaN,'C',"['Business', 'Accuracy', 'Development', 'Marke...",9


In [3]:
df_skills = pd.read_csv('skills_db2/skill.csv')
SKILLS = df_skills['skill'].unique().tolist()
len(SKILLS)

3036

In [4]:
df_redskills = pd.read_excel('skills/Redundant Skills.xlsx')
RED_SKILLS = df_redskills['Skill'].unique().tolist()
len(RED_SKILLS)

37

In [129]:
def print_job(df, i):
    row = df.loc[i]
    print('Title:', row['title'])
    print('Description:', row['description'])
    print('Indeed Skills:', row['indeed_skills'])
    
def test_extract(df, i):
    row = df.loc[i]
    info = row['title'] + ' ' + row['description']
    all_skills = extract_skills(info)
    job_skills, ignore_skills = extract_ignore(all_skills)
    job_skills.sort()
    ignore_skills.sort()
    print('Skills:', job_skills)
    print()
    print('Ignore:', ignore_skills)

def test_skill(df, i, skill, threshold=0.9):
    row = df.loc[i]
    info = row['title'] + ' ' + row['description']
    words, bigrams, trigrams = clean_info(info)
    results = []
    s = skill.lower()
    print('Unigram: {}'.format(gcm(s, words, cutoff=threshold)))
    print('Bigram: {}'.format(gcm(s, bigrams, cutoff=threshold)))
    print('Trigram: {}'.format(gcm(s, trigrams, cutoff=threshold)))
    
def extract_skills(info, threshold=0.9):
    words, bigrams, trigrams = clean_info(info)
    results = []
    for skill in SKILLS:
        s = skill.lower()
        if '(' in s:
            abb = s[s.find("(")+1:s.find(")")]
            if abb in words:
                results.append(skill)
                continue
            s = re.sub(r"[\(].*?[\)]", "", s)
        s2 = s.split()
        if len(s2) == 1:
            if len(gcm(s, words, cutoff=threshold)) > 0:
                results.append(skill)
        elif len(s2) == 2:
            if len(gcm(s, bigrams, cutoff=threshold)) > 0:
                results.append(skill)
        elif len(s2) == 3:
            if len(gcm(s, trigrams, cutoff=threshold)) > 0:
                results.append(skill)
        else:
            if len(gcm(s, trigrams, cutoff=threshold)) > 0:
                results.append(skill)
    return results

def extract_ignore(skills):
    ignore_skills = []
    for j, skill in enumerate(skills):
        if skill in RED_SKILLS:
            ignore_skills.append(skill)
        else:
            for other in skills[:j] + skills[j+1:]:
                if skill in other:
                    if find_whole_word(skill, other):
                        ignore_skills.append(skill)
                        break
    job_skills = [s for s in skills if s not in ignore_skills]
    return job_skills, ignore_skills

def clean_info(info):
    # Remove ordered list with alphabets: a), b), c),...
    words = re.sub(r'[\s\t\n]+[a-zA-Z\s*]\)+', ' ', info)
    words = re.sub('[\n|,|.|\-|/|\(|\)]', ' ', words).lower().split()
    #words = re.findall(r'[a-zA-Z](?:[A-Z]*(?![a-z])|[a-z]*)', words)
    words = [word.strip() for word in words]
    bigrams = [' '.join(g) for g in ngrams(words, 2)]
    trigrams = [' '.join(g) for g in ngrams(words, 3)]
    return words, bigrams, trigrams

def check_skill(skill):
    return df_skills.loc[df_skills['skill'].str.contains(skill)]

def find_whole_word(search_string, input_string):
    raw_search_string = r"\b" + search_string + r"\b"
    match_output = re.search(raw_search_string, input_string)
    no_match_was_found = ( match_output is None )
    if no_match_was_found:
        return False
    else:
        return True

In [109]:
job_no = random.randint(0, len(df_ds))
job_no

897

In [110]:
print_job(df_ds, job_no)

Title: #SGUnitedTraineeships Software Developer (RA Apprentice / PhD Apprentice / E&P) - Cloud #SGUP
Description: Job Description:This position reports to the Mentor / Software Development Lead and interfaces with clinicians, nurses, software development cohort, and principal investigator / project manager. NUHS is committed to an employee-orientated, high performance culture that emphasises empowerment, quality, continuous improvement, and the recruitment and on-going development of a superior workplaceThe trainees will gain exposure to the following functional areas:Latest software development platform e.g. containerized platform, cloud development platform, Big Data and machine learning analytics engines/platforms.Use of various AI models and frameworks in solving clinical problems.On the job training by Mentor, software development lead and data scientist.Other Training and Development programs on system, platforms, and engines.Specific Responsibilities:Develop software using huge 

In [130]:
test_extract(df_ds, job_no)

Skills: ['Algorithm', 'Artificial Intelligence (AI)', 'Big Data', 'C++', 'Chatbot', 'Cloud Application', 'Coding', 'Communication', 'Computer Science', 'Data Structure', 'Database Management System', 'IPython', 'Java', 'Machine Learning', 'MySQL', 'Platform', 'Programming Language', 'Python', 'Relational Database', 'Software Design', 'Software Development Life Cycle (SDLC)', 'Testing', 'Training']

Ignore: ['Application', 'Culture', 'Data', 'Database', 'Design', 'Development', 'Implementation', 'Language', 'Management', 'Programming', 'Software Development']


In [131]:
test_skill(df_ds, job_no, 'Software Development Life Cycle')

Unigram: []
Bigram: []
Trigram: ['software development lifecycle']


In [105]:
check_skill('Modeling')

,skill_id,skill
185,186,Financial Risk Modeling
474,475,3D Modeling Software
818,819,Modeling
819,820,Modeling Language
1139,1140,Bayesian Modeling
1212,1213,Data Modeling
1264,1265,Financial Modeling
1423,1424,Predictive Modeling
1520,1521,Statistical Modeling
1556,1557,Topic Modeling
